# क्लाउडमा डाटा साइन्स: "Azure ML SDK" तरिका

## परिचय

यस नोटबुकमा, हामी Azure ML SDK प्रयोग गरेर कसरी मोडेललाई प्रशिक्षण, परिनियोजन, र उपभोग गर्ने भन्ने कुरा सिक्नेछौं।

पूर्व-आवश्यकताहरू:
1. तपाईंले Azure ML कार्यक्षेत्र (Workspace) सिर्जना गर्नुभएको छ।
2. तपाईंले [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML मा लोड गर्नुभएको छ।
3. तपाईंले यो नोटबुक Azure ML स्टुडियोमा अपलोड गर्नुभएको छ।

अर्को चरणहरू:

1. अवस्थित कार्यक्षेत्रमा एउटा प्रयोग (Experiment) सिर्जना गर्नुहोस्।
2. एउटा कम्प्युट क्लस्टर सिर्जना गर्नुहोस्।
3. डाटासेट लोड गर्नुहोस्।
4. AutoMLConfig प्रयोग गरेर AutoML कन्फिगर गर्नुहोस्।
5. AutoML प्रयोग परीक्षण (Experiment) चलाउनुहोस्।
6. नतिजाहरू अन्वेषण गर्नुहोस् र उत्कृष्ट मोडेल प्राप्त गर्नुहोस्।
7. उत्कृष्ट मोडेल दर्ता गर्नुहोस्।
8. उत्कृष्ट मोडेल परिनियोजन गर्नुहोस्।
9. अन्त बिन्दु (Endpoint) उपभोग गर्नुहोस्।

## Azure Machine Learning SDK-विशेष आयातहरू


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## कार्यक्षेत्र आरम्भ गर्नुहोस्  
सङ्ग्रहित कन्फिगरेसनबाट कार्यक्षेत्र वस्तु आरम्भ गर्नुहोस्। सुनिश्चित गर्नुहोस् कि कन्फिग फाइल .\config.json मा उपस्थित छ।  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML प्रयोग गरेर एउटा प्रयोग (Experiment) बनाउनुहोस्

हामीले भर्खरै आरम्भ गरेको कार्यक्षेत्रमा 'aml-experiment' नामको एउटा प्रयोग बनाउँ।


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## कम्प्युट क्लस्टर बनाउनुहोस्  
तपाईंको AutoML रनको लागि [कम्प्युट टार्गेट](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) बनाउन आवश्यक छ।  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## डाटा  
निश्चित गर्नुहोस् कि तपाईंले डाटासेट Azure ML मा अपलोड गर्नुभएको छ र यसको कुञ्जी डाटासेटको नामसँग उस्तै छ।  


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## उत्कृष्ट मोडेल परिनियोजन गर्नुहोस्

उत्कृष्ट मोडेल परिनियोजन गर्न निम्न कोड चलाउनुहोस्। तपाईंले Azure ML पोर्टलमा परिनियोजनको अवस्था हेर्न सक्नुहुन्छ। यो चरण पूरा हुन केही मिनेट लाग्न सक्छ।


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## अन्त बिन्दु प्रयोग गर्नुहोस्
तपाईं तलको इनपुट नमुनामा इनपुटहरू थप्न सक्नुहुन्छ।


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
